In [2]:
# Import required libraries for RAG system setup
import os
import warnings
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from backend.tools.pinecone_search_tool import PineconeSearchTool
from backend.config.settings import PSU_WEBSITE_INDEX
from crewai import Agent, Task, Crew

# Suppress warnings and load environment variables
warnings.filterwarnings('ignore')
load_dotenv()

print("Setup complete")

Setup complete


In [3]:
# Initialize LLM and Pinecone search tool
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4o-mini", temperature=0.7)
pinecone_search = PineconeSearchTool(index_name=PSU_WEBSITE_INDEX)

# Wrapper function that formats query input for PineconeSearchTool
# CrewAI passes raw strings, so this function ensures proper structure: {"query": ..., "k": 5}
def search_psu_knowledge_base(query: str) -> str:
    """Wrapper that properly formats input for PineconeSearchTool."""
    return pinecone_search._run({"query": query, "k": 5})

# Wrap PineconeSearchTool with LangChain Tool for CrewAI compatibility
# CrewAI requires tools to have name, func, and description attributes
search_tool = Tool(
    name="PSU_Search",
    func=search_psu_knowledge_base,
    description="Search PSU knowledge base using Pinecone vector database. Takes a query string and returns relevant documents with sources."
)

print(f"LLM initialized: gpt-4o-mini")

print(f"Pinecone Index: {PSU_WEBSITE_INDEX}")
print(f"Search Tool: {search_tool.name}")

LLM initialized: gpt-4o-mini
Pinecone Index: psu-web-auto
Search Tool: PSU_Search


In [4]:
# Create a retrieval agent with specialized role for information search
# Agent uses the search_tool to retrieve documents from PSU knowledge base
agent = Agent(
    role="Information Retrieval Specialist",
    goal="Retrieve relevant information from PSU knowledge base",
    backstory="Expert at searching and retrieving information from university databases.",
    tools=[search_tool],
    llm=llm,
    verbose=True
)

print("Agent created and ready for queries")

Agent created and ready for queries


In [5]:
# Test the RAG system with sample queries
# Each query is processed by the agent which searches the PSU knowledge base
queries = [
    "What are the admission requirements for undergrads?",
    "Tell me about computer science programs",
    "What is the application deadline?"
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    # Create a task for the agent to search for information about the query
    task = Task(
        description=f"Search for: {query}",
        agent=agent,
        expected_output="Retrieved information"
    )
    
    # Execute the task using the crew framework (agent + task execution)
    crew = Crew(agents=[agent], tasks=[task], verbose=False)
    result = crew.kickoff()

    print()

    # Display the retrieved results    print(f"\nResult:\n{result}")


Query: What are the admission requirements for undergrads?
# Agent: Information Retrieval Specialist
## Task: Search for: What are the admission requirements for undergrads?


# Agent: Information Retrieval Specialist
## Thought: I need to search the PSU knowledge base for the admission requirements for undergraduate students.
## Using tool: PSU_Search
## Tool Input: 
"{\"query\": \"What are the admission requirements for undergrads?\"}"
## Tool Output: 
[1] College of Computer & Information Science (CCIS) || Prince Sultan University | I.	Admission to the Preparatory Year Program (PYP) | chunk=1.0
Source: https://psu.edu.sa/en/CCIS/se-entry-req
Section: I.	Admission to the Preparatory Year Program (PYP)

The Preparatory Year Program (PYP) aims to prepare students for their undergraduate study, enhance their English proficiency level, and reinforce the student’s previous knowledge of the mathematical and analytical fields in the medium of English before starting their undergraduate stu

Overriding of current TracerProvider is not allowed




# Agent: Information Retrieval Specialist
## Final Answer: 
The admission requirements for undergraduate students at Prince Sultan University (PSU) are as follows:

1. **Admission to the Preparatory Year Program (PYP)**:
   - Applicants must have a general Saudi secondary school (high school) certificate (natural sciences) or an equivalent qualification approved by the Ministry of Education in Saudi Arabia for diplomas earned outside the Kingdom.
   - Pass the University Entrance Exam.
   - Be a full-time student; part-time enrollment is not permitted.
   - If employed in the public or private sector, obtain written approval to study from the employer.
   - Pass a medical check-up, requiring a medical report filled and signed by a hospital or medical center.
   - High school diploma must have been earned within the past five years.
   - Academic aptitude is evaluated based on the following:
     - 40% from the secondary school average,
     - 30% from the Qiyas General Aptitude Test 

Overriding of current TracerProvider is not allowed




# Agent: Information Retrieval Specialist
## Final Answer: 
1. **Bachelor of Science in Computer Science**: Computer Science (CS) is a rapidly changing field. The ideas and research results emanating from it are swiftly transformed into practice. Therefore, the Bachelor of Science program in computer science offers not only technical instruction on the frontier of new developments but also provides students with a broad education that covers the core areas of computer science. This ensures a foundation for continued and sustainable career growth in the computing profession for our students. This curriculum in Computer Science is deliberately oriented towards giving the students education in the broad area of computer science from which the student can acquire the necessary skills and experience needed to solve real-world practical problems. In addition, students are given the freedom to create their own computer science study program in either breadth or depth so as to enable them to